In [1]:
import warnings
import pandas as pd 
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
import numpy as np

# Importing the SimpleImputer class
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('signal-data.csv')
df.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [4]:
df.shape
# here PAss/Fail is the target column
#Target column “ –1” corresponds to a pass 
#and “1” corresponds to a fail and the data time stamp is for that specific test point.

(1567, 592)

In [3]:
# check for invalid data and treat it
df.isna().sum()

Time          0
0             6
1             7
2            14
3            14
             ..
586           1
587           1
588           1
589           1
Pass/Fail     0
Length: 592, dtype: int64

In [24]:
df.info()
#df[df['581'] is np.nan]
#df['581'].unique()
df_dd = df.drop(['Time'], axis= 1)
y= df['Pass/Fail'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [25]:
# lets impute the data and replace NaN with  median

df_signal= df_dd.fillna(df_dd.median())

In [26]:
df_signal.isna().sum()

0            0
1            0
2            0
3            0
4            0
            ..
586          0
587          0
588          0
589          0
Pass/Fail    0
Length: 591, dtype: int64

In [27]:
uni= df_signal.nunique()
type(uni)

pandas.core.series.Series

In [8]:
uni.index

NameError: name 'uni' is not defined

In [28]:
unq_1 =[]
unq_2 =[]

for i in range(0,589):
    #print(i,uni[i])
    if(uni[i]==1):
        #print(i,uni[i])
        unq_1.append(str(i))
    if(uni[i]==2):
        #print(i,uni[i])
        unq_2.append(str(i))
print(unq_1)  
print('Total records with 1 unique value',len(unq_1) ) 
print(unq_2)  
print('Total records with 2 unique value',len(unq_2) ) 
# 
# this shows there are are 116 columns, where it has only single value, either 1, 0, 100 or as such. 
#Thus these will not have any impact of the target variable. These can be deleted directly

['5', '13', '42', '49', '52', '69', '97', '141', '149', '178', '179', '186', '189', '190', '191', '192', '193', '194', '226', '229', '230', '231', '232', '233', '234', '235', '236', '237', '240', '241', '242', '243', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '276', '284', '313', '314', '315', '322', '325', '326', '327', '328', '329', '330', '364', '369', '370', '371', '372', '373', '374', '375', '378', '379', '380', '381', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '414', '422', '449', '450', '451', '458', '461', '462', '463', '464', '465', '466', '481', '498', '501', '502', '503', '504', '505', '506', '507', '508', '509', '512', '513', '514', '515', '528', '529', '530', '531', '532', '533', '534', '535', '536', '537', '538']
Total records with 1 unique value 116
['74', '206', '209', '342', '347', '478']
Total records with 2 unique value 6


In [29]:
df_signal=df_signal.drop(unq_1, axis=1)


In [30]:
for i in range(len(unq_2)):
    #print('Total records with 1 unique value',(unq_2[i])) 
    print(df_signal[unq_2[i]].unique())
# these columns have 2 distinct values either 0 or other numeric value

[0.     4.1955]
[0. 2.]
[ 0.   46.15]
[0.     0.4472]
[ 0.     13.9147]
[  0. 200.]


In [31]:
df_signal.head()

,0,1,2,3,4,6,7,8,9,10,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,97.6133,0.1242,1.5005,0.0162,-0.0034,...,72.2889,0.5005,0.0118,0.0035,2.3630,0.0205,0.0148,0.0046,71.9005,-1
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.4966,-0.0005,-0.0148,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,95.4878,0.1241,1.4436,0.0041,0.0013,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,104.2367,0.1217,1.4882,-0.0124,-0.0033,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5031,-0.0031,-0.0072,...,72.2889,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [14]:
# check for duplicate data 
df_signal.duplicated().sum()

0

In [58]:
df_signal.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,1567.0,3014.441551,73.480841,2743.2400,2966.66500,3011.4900,3056.5400,3356.3500
1,1567.0,2495.866110,80.228143,2158.7500,2452.88500,2499.4050,2538.7450,2846.4400
2,1567.0,2200.551958,29.380973,2060.6600,2181.09995,2201.0667,2218.0555,2315.2667
3,1567.0,1395.383474,439.837330,0.0000,1083.88580,1285.2144,1590.1699,3715.0417
4,1567.0,4.171281,56.103721,0.6815,1.01770,1.3168,1.5188,1114.5366
...,...,...,...,...,...,...,...,...
586,1567.0,0.021458,0.012354,-0.0169,0.01345,0.0205,0.0276,0.1028
587,1567.0,0.016474,0.008805,0.0032,0.01060,0.0148,0.0203,0.0799
588,1567.0,0.005283,0.002866,0.0010,0.00330,0.0046,0.0064,0.0286
589,1567.0,99.652345,93.864558,0.0000,44.36860,71.9005,114.7497,737.3048


In [59]:
scaler =StandardScaler()
scaled = scaler.fit_transform(df_signal)

In [18]:
df_scaled =pd.DataFrame(scaled)
df_scaled.head()
df_scaled.describe().transpose()
# since default degree of z score is 0 we will consider both z score and standard scaler output same, 
#which is also proved in subsequent lines of code. Lets use scaled dataset to treat outliers

,count,mean,std,min,25%,50%,75%,max
0,1567.0,1.626934e-15,1.000319,-3.691957,-0.650398,-0.040180,0.573100,4.654514
1,1567.0,-5.044111e-15,1.000319,-4.203310,-0.535907,0.044124,0.534633,4.371107
2,1567.0,-5.769228e-15,1.000319,-4.762831,-0.662273,0.017525,0.595934,3.905635
3,1567.0,-4.202835e-16,1.000319,-3.173512,-0.708437,-0.250557,0.443002,5.275584
4,1567.0,-3.422951e-17,1.000319,-0.062222,-0.056228,-0.050895,-0.047293,19.797614
...,...,...,...,...,...,...,...,...
469,1567.0,-9.835782e-17,1.000319,-0.523002,-0.212858,-0.086689,0.063573,26.913337
470,1567.0,2.248078e-16,1.000319,-3.105818,-0.648390,-0.077554,0.497330,6.586246
471,1567.0,1.095964e-17,1.000319,-1.508045,-0.667329,-0.190165,0.434692,7.205869
472,1567.0,-1.765942e-16,1.000319,-1.494750,-0.692040,-0.238334,0.389874,8.137772


In [19]:
# we see that data variation is huge , thus need scaling. we will use z score to standardise the data 
#from scipy.stats import zscore
#df_signal_z=df_signal.apply(zscore)
#df_signal.describe().transpose()

In [60]:
print(' Total Pass cases',(df_signal.loc[(df_signal['Pass/Fail']== -1)]).count())
print(' Total Fail cases',(df_signal.loc[(df_signal['Pass/Fail']== 1)]).count())

## we see that data is heavyly biased towards pass cases 

 Total Pass cases 0            1463
1            1463
2            1463
3            1463
4            1463
             ... 
586          1463
587          1463
588          1463
589          1463
Pass/Fail    1463
Length: 475, dtype: int64
 Total Fail cases 0            104
1            104
2            104
3            104
4            104
            ... 
586          104
587          104
588          104
589          104
Pass/Fail    104
Length: 475, dtype: int64


In [20]:
#df_signal_z.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,1567.0,1.626934e-15,1.000319,-3.691957,-0.650398,-0.040180,0.573100,4.654514
1,1567.0,-5.044111e-15,1.000319,-4.203310,-0.535907,0.044124,0.534633,4.371107
2,1567.0,-5.769228e-15,1.000319,-4.762831,-0.662273,0.017525,0.595934,3.905635
3,1567.0,-4.202835e-16,1.000319,-3.173512,-0.708437,-0.250557,0.443002,5.275584
4,1567.0,-3.422951e-17,1.000319,-0.062222,-0.056228,-0.050895,-0.047293,19.797614
...,...,...,...,...,...,...,...,...
585,1567.0,-9.835782e-17,1.000319,-0.523002,-0.212858,-0.086689,0.063573,26.913337
586,1567.0,2.248078e-16,1.000319,-3.105818,-0.648390,-0.077554,0.497330,6.586246
587,1567.0,1.095964e-17,1.000319,-1.508045,-0.667329,-0.190165,0.434692,7.205869
588,1567.0,-1.765942e-16,1.000319,-1.494750,-0.692040,-0.238334,0.389874,8.137772


In [21]:
# if the z score is more than 3 , as per stats concepts, let us treat it as outlier
out_data =[]
out_indx =[]
out_data =df_scaled[0]
#print(np.where(out_data >3))
#print('record at  ',df_scaled.iloc[101:102,0:1])
#print(' reocrd at col 0 row 101',out_data[101])
for i in range(0,2):
    d= str(i)
    na_o = 'out_data_'+ str(i)
    print(df_scaled[i])
    na_o =df_scaled[i]
    #print(np.where(out_data >3))
na_o

0       0.224463
1       1.107287
2      -1.114000
3      -0.350156
4       0.242296
          ...   
1562   -1.565963
1563    0.515516
1564   -0.485064
1565   -1.627087
1566   -0.946420
Name: 0, Length: 1567, dtype: float64
0       0.849523
1      -0.383106
2       0.798901
3      -0.199072
4       0.087328
          ...   
1562   -0.392832
1563    0.332706
1564   -1.447412
1565    0.450658
1566   -0.562403
Name: 1, Length: 1567, dtype: float64


0       0.849523
1      -0.383106
2       0.798901
3      -0.199072
4       0.087328
          ...   
1562   -0.392832
1563    0.332706
1564   -1.447412
1565    0.450658
1566   -0.562403
Name: 1, Length: 1567, dtype: float64

In [22]:
#df_scaled.iloc[101:104,:]
#df_scaled.head(312)
#out_data = pd.DataFrame(out_data)
tts= df_scaled[0]
tts[395]

3.138669164251478

In [23]:

out_data.iloc[101]
#tts.head(103)

0.7631423904323035

In [24]:
out_data.head(103)

0      0.224463
1      1.107287
2     -1.114000
3     -0.350156
4      0.242296
         ...   
98    -0.040180
99    -1.378916
100    0.896009
101    0.763142
102   -0.738681
Name: 0, Length: 103, dtype: float64